In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import math



#ignore log(0) and divide by 0 warning
np.seterr(divide = 'ignore');
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# settings to display all columns
pd.set_option("display.max_columns", None)

print('Libraries read!')


Libraries read!


In [3]:
base_dir = '../'

# Reading  dataset

In [4]:
train_data = pd.read_excel(
    base_dir + 
    'data/previous/output/train_data_final.xlsx') 
print('Shape: ', train_data.shape)
train_data.head()

Shape:  (7823, 557)


,IDTRAIN,IDTRAIN_LOT,IDTRAIN_JALON_origin,IDGARE_origin,Code_origin,IDGARE_Parent_origin,IDTRAIN_JALON_destination,IDGARE_destination,Code_destination,IDGARE_Parent_destination,Train_num,IDTRAIN_Parent,IDTRAIN_ETAPE,Origin_Station,Origin_Country,Origin_Latitude,Origin_Longitude,Planned_Departure,Actual_Departure,Destination_Station,Destination_Country,Destination_Latitude,Destination_Longitude,Planned_Arrival,Actual_Arrival,Journey_Distance,Incoterm,Max_TEU,TEU_Count,Max_Length,Train_Length,Train_Weight,Cancellation_Reason,French_Cancel_Comments,English_Cancel_Comments,Origin_City_Country,Destination_City_Country,OD_Pair,Arc,Depart_Variance,Arrive_Variance,Depart_Variance_Mins,Depart_Variance_Mins_Pos,Arrive_Variance_Mins,Arrive_Variance_Mins_Pos,Late_Departure_Bin,Late_Arrival_Bin,Late_Departure_Bin_15,Late_Arrival_Bin_15,Planned_Departure_Date,Planned_Arrival_Date,Planned_Departure_DOW,Planned_Arrival_DOW,Depart_Week_Num,Actual_Departure_Bin,Actual_Arrival_Bin,BB_Departure_Bin,Cancelled_Train_Bin,wagon_count,Train_Distance_KM,Train_Total_Time_Mins,Train_Travel_Time_Mins,Train_Idle_Time_Mins,Travel_KM_HR,Total_KM_HR,Station_Name_0,Station_Name_1,Station_Name_2,Station_Name_3,Station_Name_4,Station_Name_5,Station_Name_6,Station_Name_7,Station_Name_8,Station_Name_9,Station_Name_10,Station_Name_11,Station_Name_12,Station_Name_13,Station_Name_14,Station_Name_15,Station_Name_16,Station_Name_17,Station_Name_18,Station_Name_19,Station_Name_20,Station_Name_21,Station_Name_22,Station_Name_23,Plan_Timestamp_0,Plan_Timestamp_1,Plan_Timestamp_2,Plan_Timestamp_3,Plan_Timestamp_4,Plan_Timestamp_5,Plan_Timestamp_6,Plan_Timestamp_7,Plan_Timestamp_8,Plan_Timestamp_9,Plan_Timestamp_10,Plan_Timestamp_11,Plan_Timestamp_12,Plan_Timestamp_13,Plan_Timestamp_14,Plan_Timestamp_15,Plan_Timestamp_16,Plan_Timestamp_17,Plan_Timestamp_18,Plan_Timestamp_19,Plan_Timestamp_20,Plan_Timestamp_21,Plan_Timestamp_22,Plan_Timestamp_23,Actual_Timestamp_0,Actual_Timestamp_1,Actual_Timestamp_2,Actual_Timestamp_3,Actual_Timestamp_4,Actual_Timestamp_5,Actual_Timestamp_6,Actual_Timestamp_7,Actual_Timestamp_8,Actual_Timestamp_9,Actual_Timestamp_10,Actual_Timestamp_11,Actual_Timestamp_12,Actual_Timestamp_13,Actual_Timestamp_14,Actual_Timestamp_15,Actual_Timestamp_16,Actual_Timestamp_17,Actual_Timestamp_18,Actual_Timestamp_19,Actual_Timestamp_20,Actual_Timestamp_21,Actual_Timestamp_22,Actual_Timestamp_23,Time_From_Prior_Plan_Mins_0,Time_From_Prior_Plan_Mins_1,Time_From_Prior_Plan_Mins_2,Time_From_Prior_Plan_Mins_3,Time_From_Prior_Plan_Mins_4,Time_From_Prior_Plan_Mins_5,Time_From_Prior_Plan_Mins_6,Time_From_Prior_Plan_Mins_7,Time_From_Prior_Plan_Mins_8,Time_From_Prior_Plan_Mins_9,Time_From_Prior_Plan_Mins_10,Time_From_Prior_Plan_Mins_11,Time_From_Prior_Plan_Mins_12,Time_From_Prior_Plan_Mins_13,Time_From_Prior_Plan_Mins_14,Time_From_Prior_Plan_Mins_15,Time_From_Prior_Plan_Mins_16,Time_From_Prior_Plan_Mins_17,Time_From_Prior_Plan_Mins_18,Time_From_Prior_Plan_Mins_19,Time_From_Prior_Plan_Mins_20,Time_From_Prior_Plan_Mins_21,Time_From_Prior_Plan_Mins_22,Time_From_Prior_Plan_Mins_23,Depart_Variance_Mins_0,Depart_Variance_Mins_1,Depart_Variance_Mins_2,Depart_Variance_Mins_3,Depart_Variance_Mins_4,Depart_Variance_Mins_5,Depart_Variance_Mins_6,Depart_Variance_Mins_7,Depart_Variance_Mins_8,Depart_Variance_Mins_9,Depart_Variance_Mins_10,Depart_Variance_Mins_11,Depart_Variance_Mins_12,Depart_Variance_Mins_13,Depart_Variance_Mins_14,Depart_Variance_Mins_15,Depart_Variance_Mins_16,Depart_Variance_Mins_17,Depart_Variance_Mins_18,Depart_Variance_Mins_19,Depart_Variance_Mins_20,Depart_Variance_Mins_21,Depart_Variance_Mins_22,Depart_Variance_Mins_23,Arrive_Variance_Mins_0,Arrive_Variance_Mins_1,Arrive_Variance_Mins_2,Arrive_Variance_Mins_3,Arrive_Variance_Mins_4,Arrive_Variance_Mins_5,Arrive_Variance_Mins_6,Arrive_Variance_Mins_7,Arrive_Variance_Mins_8,Arrive_Variance_Mins_9,Arrive_Variance_Mins_10,Arrive_Variance_Mins_11,Arrive_Variance_Mins_12,Arrive_Variance_Mins_13,Arrive_Variance_Mins_

In [5]:
station_columns = [col for col in train_data.columns if 'Station_Name' in col]
routes_data = train_data[station_columns]
routes_data = routes_data.drop_duplicates(keep='first').reset_index(drop=True)
routes_data = routes_data.replace({np.nan: 0})
routes_data

,Station_Name_0,Station_Name_1,Station_Name_2,Station_Name_3,Station_Name_4,Station_Name_5,Station_Name_6,Station_Name_7,Station_Name_8,Station_Name_9,Station_Name_10,Station_Name_11,Station_Name_12,Station_Name_13,Station_Name_14,Station_Name_15,Station_Name_16,Station_Name_17,Station_Name_18,Station_Name_19,Station_Name_20,Station_Name_21,Station_Name_22,Station_Name_23
0,Zeebrugge,Gent,Gent,Muizen,Muizen,Ronet,Ronet,BERTRIX,BERTRIX,Rodange-Athus-frontière,Rodange-Athus-frontière,Bettembourg,0,0,0,0,0,0,0,0,0,0,0,0
1,Bettembourg,Wasserbillig,Wasserbillig,Ehrang,Ehrang,Saarbrücken Rbf,Saarbrücken Rbf,München-Laim,München-Laim,Salzburg-Hbf,Salzburg-Hbf,Tarvisio-Boscoverde,Tarvisio-Boscoverde,Trieste,0,0,0,0,0,0,0,0,0,0
2,Bettembourg,Boulou-Perthus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bettembourg,Hagondange,Hagondange,Metz-Sablon,Metz-Sablon,Champigneulles,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Boulou-Perthus,Bettembourg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Rostock,Schwerin-Goerris,Schwerin-Goerris,Hamburg Harburg,Hamburg Harburg,Kirchweyhe,Kirchweyhe,Osnabrück,Osnabrück,Duisburg-Wedau,Duisburg-Wedau,Koblenz Mosel Gbf,Koblenz Mosel Gbf,Ehrang,Ehrang,Apach-Frontiere,Apach-Frontiere,Metz-Sablon,Metz-Sablon,Metz-Chambiere,Metz-Chambiere,Zoufftgen-Frontiere,Zoufftgen-Frontiere,Bettembourg
164,Trieste,Grignano,Grignano,Tarvisio-Boscoverde,Tarvisio-Boscoverde,Salzburg Hbf,Salzburg Hbf,München-Laim,München-Laim,Saarbrücken Rbf,Saarbrücken Rbf,Apach-Frontiere,Apach-Frontiere,Hagondange,Hagondange,Zoufftgen-Frontiere,Zoufftgen-Frontiere,Bettembourg,0,0,0,0,0,0
165,Trieste,Grignano,Grignano,Tarvisio-Boscoverde,Tarvisio-Boscoverde,Salzburg Hbf,Salzburg Hbf,München-Laim,München-Laim,Saarbrücken Rbf,Saarbrücken Rbf,Forbach,Forbach,Zoufftgen-Frontiere,Zoufftgen-Frontiere,Bettembourg,0,0,0,0,0,0,0,0
166,Bettembourg,Zoufftgen-Frontiere,Zoufftgen-Frontiere,Metz-Sablon,Metz-Sablon,Forbach-Frontiere,Forbach-Frontiere,Saarbrücken Rbf,Saarbrücken Rbf,München-Laim,München-Laim,Salzburg Hbf,Salzburg Hbf,Tarvisio-Boscoverde,Tarvisio-Boscoverde,Trieste,0,0,0,0,0,0,0,0


In [6]:
routes = pd.DataFrame()
routes['stations'] = [list(x) for x in zip(*[routes_data[col] for col in routes_data])]

routes['stations'] = routes['stations'].apply(lambda x: list(dict.fromkeys(x)))

for index, row in routes.iterrows():
    row['stations'] = [x for x in row['stations'] if x != 0]

routes


,stations
0,"[Zeebrugge, Gent, Muizen, Ronet, BERTRIX, Roda..."
1,"[Bettembourg, Wasserbillig, Ehrang, Saarbrücke..."
2,"[Bettembourg, Boulou-Perthus]"
3,"[Bettembourg, Hagondange, Metz-Sablon, Champig..."
4,"[Boulou-Perthus, Bettembourg]"
...,...
163,"[Rostock, Schwerin-Goerris, Hamburg Harburg, K..."
164,"[Trieste, Grignano, Tarvisio-Boscoverde, Salzb..."
165,"[Trieste, Grignano, Tarvisio-Boscoverde, Salzb..."
166,"[Bettembourg, Zoufftgen-Frontiere, Metz-Sablon..."


In [7]:
routes.to_csv(base_dir + 
    'data/previous/output/routes.csv', index=False)